In [4]:
#path = "/data/Radarbilder_DWD/TEST/"  # Pfad zu den Bilddaten
#max_num_samples = 2                   # Maximale anzahl Samples (kann auch -1 sein, für alle einlesen)
#n_data = 1                            # Anzahl Zeitschritte (t) für die Eingangsdaten (x,y,t) bei n_data=3 werden also

import os
# data=os.listdir(os.getcwd())

In [5]:
path_image_dir="/data/Radarbilder_DWD/minutely/2016"
#os.path.exists(path_image_dir)
image_data=os.listdir(path_image_dir)
#print(image_data)

In [6]:
wr = open("date_and_rainfall_hourly_june16.txt","r")

rain_liste = []
datum_liste = []

for line in wr:
    a = line.split(" ")
    datum = a[0]
    value = a[1]
    rain_liste.append(value)
    datum_liste.append(datum)
    
wr.close()

laenge= len(datum_liste)

In [7]:
rain_liste.reverse()

import numpy as np

image_matrix = np.zeros((laenge + 1,80), dtype='object')
#print(image_matrix)
index = 1

for element in datum_liste:
    image_matrix[index][0] = index
    image_matrix[index][1] = element
    image_matrix[index][2] = rain_liste.pop()
    index = index + 1
    
# erste zeile mit pixel füllen
spalte = 4
for i in range(373, 381):
    for j in range(153, 161):
        image_matrix[0][spalte] = j, i
        if spalte < 79:
            spalte = spalte + 1
        else: 
            break
#for line in image_matrix:
  #  print(line)

In [8]:
from PIL import Image #from IPython.core.debugger import set_trace dann   #set_trace() im code als breakpoint
spalte = 4   
for image in image_data:
    if(image[9] == '0' and image[10] == '6' and image[15] == '0' and image[16] == '0'):
        for zeile in range(1, laenge +1):
            if (image[9] == '0' and image[10] == '6' and image[15] == '0' and image[16] == '0' 
                and image[11] == image_matrix[zeile][1][6] and image[12] == image_matrix[zeile][1][7] 
                and image[13] == image_matrix[zeile][1][8] and image[14] == image_matrix[zeile][1][9]):
                #print(image, " verglichen mit ", image_matrix[zeile][1]) 
                image_matrix[zeile][3] = 'true'

                img = Image.open("/data/Radarbilder_DWD/minutely/2016/" + image)
                pixels = img.load()
                #print('Bildhoehe: %d und Bildbreite %d' % (img.width, img.height))

                for i in range(373, 381):
                    for j in range(153, 161):   #print(i, " ", j, " ", spalte)
                        #import pdb; pdb.set_trace() # Debugger, "n" for next line, "c" continue, "h" help
                        image_matrix[zeile][spalte] = (str(pixels[j, i])) #print(str(pixels[j, i]))
                        if spalte < 78:
                            spalte = spalte + 1
                        else:
                            break
    spalte = 4
    
for zeile in range(1, laenge+1):
    if (image_matrix[zeile][3] != 'true'):
        image_matrix[zeile][3] = 'false'     
        #print(datum, ' ', image) Location (156, 377)
    
for line in image_matrix:
    print(line)

[0 0 0 0 (153, 373) (154, 373) (155, 373) (156, 373) (157, 373) (158, 373)
 (159, 373) (160, 373) (153, 374) (154, 374) (155, 374) (156, 374)
 (157, 374) (158, 374) (159, 374) (160, 374) (153, 375) (154, 375)
 (155, 375) (156, 375) (157, 375) (158, 375) (159, 375) (160, 375)
 (153, 376) (154, 376) (155, 376) (156, 376) (157, 376) (158, 376)
 (159, 376) (160, 376) (153, 377) (154, 377) (155, 377) (156, 377)
 (157, 377) (158, 377) (159, 377) (160, 377) (153, 378) (154, 378)
 (155, 378) (156, 378) (157, 378) (158, 378) (159, 378) (160, 378)
 (153, 379) (154, 379) (155, 379) (156, 379) (157, 379) (158, 379)
 (159, 379) (160, 379) (153, 380) (154, 380) (155, 380) (156, 380)
 (157, 380) (158, 380) (159, 380) (160, 380) 0 0 0 0 0 0 0 0 0 0 0 0]
[1 '201606010000' '0.0' 'false' 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 '201606010100' '0.28' 'false' 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [9]:
ww = open("rainfall_pixels_hourly_june16.txt","w")

for line in image_matrix:
    for column in line:
        ww.write(str(column))
        ww.write("\t")
    ww.write("\n")
        
ww.close()

In [ ]:
# Erste Einfache Auswertung
# Anzahl Vorhandener Daten für radar und niederschlag
count_true = 0
liste_true=[]
liste_true.append(0)

for zeile in range(1, laenge+1):
    if image_matrix[zeile][3] == 'true':
        count_true = count_true + 1
        liste_true.append(zeile)

matrix_true = np.zeros((count_true + 1,80), dtype='object')
verhaeltnis = count_true / len(image_matrix)


print("Anzahl an 'true' Werten: ", count_true, " von insgesamt: ", len(image_matrix), 
      " Verhältnis: ", round(verhaeltnis, 4) * 100, "%")

z = 0
for index in liste_true:
    
    sp = 0
    for i in range(80):
        import pdb; pdb.set_trace()
        matrix_true[z][sp] = image_matrix[index][sp]
        z = z +1
        sp=sp+1


print()

In [ ]:
# Matrix nur mit 'true'-Werten

import matplotlib as mlp
import matplotlib.pyplot as plt
